In [ ]:
import glob
import pandas as pd
import re
vote_resp = []

def extract_number(filename):
    match = re.search(r'(\d+).csv', filename)
    return int(match.group(1)) if match else 0

for file in sorted(glob.glob("resources/gpt4omini-medqa-vote/*.csv"), key=extract_number):
    df = pd.read_csv(file)
    vote_resp.append(df)



resources/gpt4omini-medqa-vote/0.csv
resources/gpt4omini-medqa-vote/1.csv
resources/gpt4omini-medqa-vote/2.csv
resources/gpt4omini-medqa-vote/3.csv
resources/gpt4omini-medqa-vote/4.csv
resources/gpt4omini-medqa-vote/5.csv
resources/gpt4omini-medqa-vote/6.csv
resources/gpt4omini-medqa-vote/7.csv
resources/gpt4omini-medqa-vote/8.csv
resources/gpt4omini-medqa-vote/9.csv
resources/gpt4omini-medqa-vote/10.csv
resources/gpt4omini-medqa-vote/11.csv
resources/gpt4omini-medqa-vote/12.csv
resources/gpt4omini-medqa-vote/13.csv
resources/gpt4omini-medqa-vote/14.csv
resources/gpt4omini-medqa-vote/15.csv
resources/gpt4omini-medqa-vote/16.csv
resources/gpt4omini-medqa-vote/17.csv
resources/gpt4omini-medqa-vote/18.csv
resources/gpt4omini-medqa-vote/19.csv
resources/gpt4omini-medqa-vote/20.csv
resources/gpt4omini-medqa-vote/21.csv
resources/gpt4omini-medqa-vote/22.csv
resources/gpt4omini-medqa-vote/23.csv
resources/gpt4omini-medqa-vote/24.csv
resources/gpt4omini-medqa-vote/25.csv
resources/gpt4omini-me

1273

In [38]:

df = pd.read_json('./resources/data/medqa.jsonl', lines=True)
letter_to_number = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5}
real_ans = df["answer_idx"].apply(lambda x: letter_to_number.get(x))

In [43]:

resp = []

def extract_number(filename):
    match = re.search(r'(\d+).csv', filename)
    return int(match.group(1)) if match else 0

for file in sorted(glob.glob("resources/gpt4omini-medqa-vote/*.csv"), key=extract_number):
    df = pd.read_csv(file).head(1)
    resp.append(df)



pred_ans = [int(df.answer.mode()[0]) for df in resp]
sum(real_ans == pred_ans) / len(real_ans)


0.7297721916732128

In [44]:

resp = []

def extract_number(filename):
    match = re.search(r'(\d+).csv', filename)
    return int(match.group(1)) if match else 0

for file in sorted(glob.glob("resources/gpt4omini-medqa-vote/*.csv"), key=extract_number):
    df = pd.read_csv(file)
    resp.append(df)



pred_ans = [int(df.answer.mode()[0]) for df in resp]
sum(real_ans == pred_ans) / len(real_ans)


0.7415553809897879

In [45]:

resp = []

def extract_number(filename):
    match = re.search(r'(\d+).csv', filename)
    return int(match.group(1)) if match else 0

for file in sorted(glob.glob("resources/gpt4omini-medqa-bayesian/*.csv"), key=extract_number):
    df = pd.read_csv(file)
    resp.append(df)



pred_ans = [int(df.answer.mode()[0]) for df in resp]
sum(real_ans == pred_ans) / len(real_ans)


0.7376276512175962

In [46]:

resp = []

def extract_number(filename):
    match = re.search(r'(\d+).csv', filename)
    return int(match.group(1)) if match else 0

for file in sorted(glob.glob("resources/gpt4omini-medqa-sbft/*.csv"), key=extract_number):
    df = pd.read_csv(file)
    resp.append(df)



pred_ans = [int(df.answer.mode()[0]) for df in resp]
sum(real_ans == pred_ans) / len(real_ans)


0.7415553809897879

In [ ]:

resp = []

def extract_number(filename):
    match = re.search(r'(\d+).csv', filename)
    return int(match.group(1)) if match else 0

for file in sorted(glob.glob("resources/gpt4omini-medqa-bayesian/*.csv"), key=extract_number):
    df = pd.read_csv(file)
    resp.append(df)



pred_ans = [int(df.answer.mode()[0]) for df in resp]
sum(real_ans == pred_ans) / len(real_ans)


In [50]:
import typing
import sys

import pydantic
import langchain_openai
import langchain_core
import tqdm.auto
import pandas as pd

from consol.confidence_models import AbstractConfidenceModel, SbftConfidenceModel, SprtConfidenceModel, PValueConfidenceModel, BayesianConfidenceModel, VoteConfidenceModel

class ConfidentSolverConfig(pydantic.BaseModel):
    max_trials: int

class ConfidentSolver:
    def __init__(
        self,
        confidence_model: typing.Union[str, AbstractConfidenceModel],
        max_trials=40,
    ):
        self.config = ConfidentSolverConfig(
            max_trials=max_trials,
        )
        if confidence_model == "sbft":
            self.confidence_model = SbftConfidenceModel()
        elif confidence_model == "sprt":
            self.confidence_model = SprtConfidenceModel()
        elif confidence_model == "pvalue":
            self.confidence_model = PValueConfidenceModel()
        elif confidence_model == "bayesian":
            self.confidence_model = BayesianConfidenceModel()
        elif confidence_model == "vote":
            self.confidence_model = VoteConfidenceModel()
        elif isinstance(confidence_model, AbstractConfidenceModel):
            self.confidence_model = confidence_model
        else:
            raise ValueError(f"Unknown Confidence Model: {confidence_model}")

    def invoke(self, answers, debug=False):
        max_trials = self.config.max_trials

        total_raw_outputs = []
        total_raw_outputs.append(answers[0]) 
        i = 1

        total_invoke = 0
        while True:     
            total_invoke += 1       
            first, second = self._get_top_two_answers(total_raw_outputs)
            trials = self._determine_trials(first, second, max_trials, len(total_raw_outputs))
            if trials == 0:
                break
            for j in range(trials):
                total_raw_outputs.append(answers[i])
                i += 1
        df = self._create_dataframe(total_raw_outputs)
        if debug:
            return df
        return {"target": df['answer'].mode().iloc[0], "total_invoke": total_invoke, "total_runs": len(total_raw_outputs)}

    def _get_top_two_answers(self, total_raw_outputs):
        total_ss = pd.Series(total_raw_outputs).value_counts()
        two = total_ss.sort_values(ascending=False).head(2).to_list()
        while len(two) < 2:
            two += [0]
        return two[0], two[1]

    def _determine_trials(self, first, second, max_trials, current_trials):
        for trials in range(0, max_trials + 1):
            if first + trials == 0:
                continue
            if self.confidence_model.test(first + trials, second):
                break
        if trials >= max_trials - current_trials:
            trials = max_trials - current_trials
        return trials

    def _create_dataframe(self, total_raw_outputs):
        return pd.DataFrame({
            'answer': total_raw_outputs,
        })

In [51]:

MAX_EXPR = 10_000
MAX_ITER = 40

records = []
for i, problem in tqdm.tqdm(enumerate(vote_resp)):
    problem = problem["answer"]
    answer = real_ans[i]

    for solver_name in ["sbft", "sprt", "pvalue","bayesian"]:
        solver = ConfidentSolver(solver_name)
        ret = solver.invoke(problem)
        ret["solver"] = solver_name
        ret["answer"] = answer
        ret["correct"] = ret["answer"] == ret["target"]
        records.append(ret)

1273it [00:05, 214.43it/s]


In [52]:
df = pd.DataFrame(records)
print(df)

      target  total_invoke  total_runs    solver  answer  correct
0        2.0             2           9      sbft       3    False
1        2.0             2           9      sprt       3    False
2        2.0             2           5    pvalue       3    False
3        2.0             2           4  bayesian       3    False
4        5.0             5          40      sbft       5     True
...      ...           ...         ...       ...     ...      ...
5087     3.0             2           4  bayesian       3     True
5088     3.0             2           9      sbft       3     True
5089     3.0             2           9      sprt       3     True
5090     3.0             2           5    pvalue       3     True
5091     3.0             2           4  bayesian       3     True

[5092 rows x 6 columns]


In [53]:
df.groupby("solver")["correct"].mean()

solver
bayesian    0.742341
pvalue      0.740770
sbft        0.741555
sprt        0.741555
Name: correct, dtype: float64

In [54]:

MAX_EXPR = 10_000
MAX_ITER = 64

records = []
for i, problem in tqdm.tqdm(enumerate(vote_resp)):
    problem = problem["answer"]
    answer = problem.mode()[0]

    for solver_name in ["sbft", "sprt", "pvalue","bayesian"]:
        solver = ConfidentSolver(solver_name)
        ret = solver.invoke(problem)
        ret["solver"] = solver_name
        ret["answer"] = answer
        ret["correct"] = ret["answer"] == ret["target"]
        records.append(ret)

1273it [00:06, 211.29it/s]


In [55]:
df = pd.DataFrame(records)
print(df)

      target  total_invoke  total_runs    solver  answer  correct
0        2.0             2           9      sbft     2.0     True
1        2.0             2           9      sprt     2.0     True
2        2.0             2           5    pvalue     2.0     True
3        2.0             2           4  bayesian     2.0     True
4        5.0             5          40      sbft     5.0     True
...      ...           ...         ...       ...     ...      ...
5087     3.0             2           4  bayesian     3.0     True
5088     3.0             2           9      sbft     3.0     True
5089     3.0             2           9      sprt     3.0     True
5090     3.0             2           5    pvalue     3.0     True
5091     3.0             2           4  bayesian     3.0     True

[5092 rows x 6 columns]


In [56]:
df.groupby("solver")["correct"].mean()

solver
bayesian    0.996072
pvalue      0.998429
sbft        1.000000
sprt        0.999214
Name: correct, dtype: float64

In [57]:
df.groupby("solver")["total_runs"].mean()

solver
bayesian     8.293794
pvalue       9.831893
sbft        16.118617
sprt        14.615082
Name: total_runs, dtype: float64